In [ ]:
%reset
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import datetime
from tsf_bmw.uvtsf import UVariateTimeSeriesForecaster
from tsf_bmw.tools import moving_linreg
from plotly.plotly import plot_mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
%matplotlib inline
from random import sample

##### Read in the data

In [ ]:
try:
    ts_univ = pd.read_csv('../data/GOOG.csv', index_col='Date', usecols=['Date','Adj Close'], parse_dates=True)
    print ( "====================================\nData: data of shape " + str(ts_univ.shape) + " has been read in." ) 
except IOError as e:
    print("Could not read in the data: {}".format(e))

In [ ]:
ts_univ.plot()

##### Create UVariateTimeSeriesForecaster

In [ ]:
uv_tsf = UVariateTimeSeriesForecaster(dict({'ts_df':ts_univ, 'time_format': "%Y-%m-%d %H:%M:%S", 'freq':'D', 'transform': 'log10',  'p_train':0.9}))
uv_tsf.ts_resample()
plt.plot(uv_tsf.ts_df)

In [ ]:
uv_tsf.ts_decompose(params=dict({'model':'additive'}))
uv_tsf.plot_decompose()
plt.plot(uv_tsf.model['seasonal'].iloc[1:20, ])
plt.gcf ().autofmt_xdate ()

In [ ]:
method='auto_arima'
dict_params = dict()
dict_params = {
    'max_p': 5,
    'max_q': 5,
    'max_P': 5,
    'max_Q':5
}

In [ ]:
method='es'
dict_params = dict()

dict_params = {
    'trend': 'additive',
    'seasonal': True,
    'seasonal_periods': 12,
    'smoothing_level': 0.2,
    'use_boxcox':True,
    'damped': False}

In [ ]:
method = 'prophet'
dict_params = {
    'interval_width': 0.95,
    'yearly_seasonality': True,
    'weekly_seasonality': False,
    'monthly_seasonality': True,
    'quarterly_seasonality': False,
    'daily_seasonality': True,
    'weekend_seasonality': True,
    'changepoint_prior_scale': 0.001,
    'consider_holidays':True,
    'country': 'US',
    'diagnose': True,
    'history': '1460 days',
    'horizon': '365 days',
    'period': '180 days'
}

In [ ]:
if len(dict_params):
    uv_tsf.ts_fit ( method=method, dict_params=dict_params ) 
else:
    uv_tsf.ts_fit ( method=method ) 

In [ ]:
uv_tsf.ts_diagnose()

In [ ]:
uv_tsf.plot_residuals()

In [ ]:
uv_tsf.ts_test()

In [ ]:
uv_tsf.ts_forecast(n_forecast = 50)